## Load libraries

In [1]:
import os
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

from alphai_watson.performance import GANPerformanceAnalysis
from alphai_watson.transformer import NullTransformer
from alphai_rickandmorty_oracle.datasource.kddcup99 import KDDCup99DataSource
from alphai_rickandmorty_oracle.detective import RickAndMortyDetective
from alphai_rickandmorty_oracle.model_kddcup99 import RickAndMorty

/opt/anaconda/envs/ai/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/opt/anaconda/envs/ai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
DEBUG:matplotlib:CACHEDIR=/home/ubuntu/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/ubuntu/.cache/matplotlib/fontList.json
DEBUG:matplotlib.backends:backend agg version v2.2


Enabling weight norm
Uppercase local vars:
	BATCH_SIZE: 50
	CRITIC_ITERS: 5
	DEFAULT_FIT_EPOCHS: 1000
	DEFAULT_LEARN_RATE: 0.0001
	DEFAULT_TRAIN_ITERS: 5000
	DEFAULT_Z_DIM: 32
	DIAGNOSIS_LEARN_RATE: 0.01
	DIM: 64
	DISC_FILTER_SIZE: 5
	INIT_KERNEL: <function variance_scaling_initializer.<locals>._initializer at 0x7fc2210c1730>
	LAMBDA: 10
	LAMBDA_2: 2.0
	OUTPUT_DIM: 121


## Define KDDCup99 Datasource

In [2]:
file_path = '../../tests/resources'
data_filename = os.path.join(file_path, 'kddcup.data_10_percent_corrected')
header_filename = os.path.join(file_path, 'kddcup.names')

kdd_datasource = KDDCup99DataSource(source_file=data_filename,
                                    header_file=header_filename,
                                    transformer=NullTransformer(8, 8))

DEBUG:root:Start file parsing.
DEBUG:root:Normal (97278, 122); Train (68095, 121); Test(29183, 121)
DEBUG:root:Abnormal (396743, 121)
DEBUG:root:End file parsing.


In [3]:
data_normal_train = kdd_datasource.get_train_data('NORMAL')
data_normal_test = kdd_datasource.get_train_data('NORMAL_TEST')
data_abnormal_test = kdd_datasource.get_train_data('ABNORMAL_TEST')

## Define Model

In [4]:
model_dir = './kddcup99_models'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

batch_size = 64
output_dimensions = 121
train_iters = 3000
plot_save_path = model_dir


model = RickAndMorty(batch_size=batch_size, 
                     output_dimensions=output_dimensions, 
                     train_iters=train_iters,
                     plot_save_path=plot_save_path)

detective = RickAndMortyDetective(model_configuration={
    'model': model,
    'batch_size': batch_size,
    'output_dimensions': output_dimensions,
    'train_iters': train_iters,
    'save_path' : '{}/KDDCup99-model'.format(model_dir),
    'plot_save_path': plot_save_path
    
})

detective.train(data_normal_train)

DEBUG:root:Starting session
DEBUG:root:Start training loop...
INFO:root:Initialising Model
INFO:root:Training iteration 0 of 3000
DEBUG:matplotlib.font_manager:findfont: Matching :family=sans-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=10.0 to DejaVu Sans ('/opt/anaconda/envs/ai/lib/python3.6/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf') with score of 0.050000


iter 0	train disc cost	1.4071215391159058	time	0.247755765914917
iter 1	train gen cost	0.6588791608810425	train disc cost	1.3958030939102173	time	0.09719562530517578
iter 2	train gen cost	0.6646692752838135	train disc cost	1.3857131004333496	time	0.06753754615783691
iter 3	train gen cost	0.6699368953704834	train disc cost	1.377500295639038	time	0.06844258308410645
iter 4	train gen cost	0.674412727355957	train disc cost	1.3695532083511353	time	0.06746673583984375


INFO:root:Saving fake samples to png.
INFO:root:Training iteration 100 of 3000


iter 99	train gen cost	1.2602397203445435	train disc cost	0.7529851198196411	time	0.06808638823659796


INFO:root:Saving fake samples to png.
INFO:root:Training iteration 200 of 3000


...

iter 2799	train gen cost	0.9468779563903809	train disc cost	1.0860507488250732	time	0.05175078868865967


INFO:root:Saving fake samples to png.
INFO:root:Training iteration 2900 of 3000


iter 2899	train gen cost	0.9473578929901123	train disc cost	1.0943915843963623	time	0.06559078454971314


INFO:root:Saving fake samples to png.
DEBUG:root:Training complete.


iter 2999	train gen cost	0.9460355639457703	train disc cost	1.0943360328674316	time	0.053946518898010255


## Evaluate results
***

### Collate ground truth

In [5]:
n1 = np.ones(len(data_normal_test.data))
n2 = np.zeros(len(data_abnormal_test.data))

ground_truth = np.hstack((n1, n2))

### Collate detection results

In [6]:
test_results_normal = detective.detect(data_normal_test).data 
test_results_abnormal = detective.detect(data_abnormal_test).data 

predictions = np.hstack((test_results_normal, test_results_abnormal))

INFO:root:Running detector on <alphai_watson.datasource.Sample object at 0x7fc1daae51d0>
INFO:root:Detection completed in 0.46583758294582367
INFO:root:Running detector on <alphai_watson.datasource.Sample object at 0x7fc1daae5278>
INFO:root:Detection completed in 5.483396312221885


### Calculate ROC Score

In [7]:
roc_score = GANPerformanceAnalysis({}).analyse(
  detection_result=predictions,
  expected_truth=ground_truth
)

print('ROC Score: {}'.format(roc_score))

ROC Score: 0.734019941461691


### Calculate training accuracy

In [8]:
def model_accuracy(data, status, threshold=None):
    results = detective.detect(data).data 
    if threshold is None:
        threshold = np.median(results)
    ground_truth = [status] * len(results)
    prediction = [1 if x >= threshold else 0 for x in results]
    print('Accuracy: {0:.2f}%'.format(100*accuracy_score(ground_truth, prediction)))
    return threshold

In [9]:
threshold = model_accuracy(data_normal_train, 1)

INFO:root:Running detector on <alphai_watson.datasource.Sample object at 0x7fc1daae5208>
INFO:root:Detection completed in 0.9801656883209944


Accuracy: 50.00%


### Generate classification report

In [10]:
class_predictions = [1 if x >= threshold else 0 for x in predictions]

target_names = ['ABNORMAL', 'NORMAL']
print(classification_report(ground_truth, class_predictions, target_names=target_names))

              precision    recall  f1-score   support

    ABNORMAL       0.96      0.93      0.95    396743
      NORMAL       0.34      0.50      0.41     29183

   micro avg       0.90      0.90      0.90    425926
   macro avg       0.65      0.72      0.68    425926
weighted avg       0.92      0.90      0.91    425926

